### Алгоритмы рекомендательных систем. Коллаборативная фильтрация (подобие по косинусу)

In [1]:
import math, random
from collections import defaultdict, Counter
def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

#
# коллаборативная фильтрация на основе пользователя
#

def cosine_similarity(v, w):
    return dot(v, w) / math.sqrt(dot(v, v) * dot(w, w))

unique_interests = sorted(list({ interest
                                 for user_interests in users_interests
                                 for interest in user_interests }))

def make_user_interest_vector(user_interests):
    """при заданном списке интересующих пользователя тем создать вектор,
    чей i-ый элемент равен 1, если unique_interests[i] есть в списке,
    и 0 в противном случае"""
    return [1 if interest in user_interests else 0
            for interest in unique_interests]

user_interest_matrix = list(map(make_user_interest_vector, users_interests))

user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_matrix]
                     for interest_vector_i in user_interest_matrix]

def most_similar_users_to(user_id):
    pairs = [(other_user_id, similarity)                      # найти других
             for other_user_id, similarity in                 # пользователей с
                enumerate(user_similarities[user_id])         # ненулевым коэфф.
             if user_id != other_user_id and similarity > 0]  # подобия

    return sorted(pairs,                                      # отсортировать их
                  key=lambda pair: pair[1],                   # сперва наиболее
                  reverse=True)                               # похожие

### Здесь примечательный момент. Мы получили наиболее похожих по косинусу пользователей, для данного пользователя `user_id`. Но как выбрать темы? Мы же не можем просто выбрать темы наиболее подходящего соседа и предложить их. Может, он сам еще не дошел до тех тем которые волнуют остальной народ. Алгоритм более интересный - он берет все темы по каждому пользователю (кроме самого `user_id`) и суммирует их коэффициенты подобия. Получается, что интересующие ближайших соседей темы получают большую сумму коэффиентов, но и с другой стороны, если тема волнует подавляющее большинство соседей, то она тоже получит большую сумму и будет рекомендована. Компромисс между известностью темы и ее наличием у ближайших соседей:

In [2]:
def user_based_suggestions(user_id, include_current_interests=False):
    # суммировать все коэффициенты подобия
    suggestions = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

    # преобразовать их в сортированный список
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    # и (может быть) исключить уже имеющиеся темы
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

### Аналогичный алгоритм, но уже не по соседям, а по темам, их схожести:

In [3]:
#
# коллаборативная фильтрация по схожести предметов
#

interest_user_matrix = [[user_interest_vector[j]
                         for user_interest_vector in user_interest_matrix]
                        for j, _ in enumerate(unique_interests)]

interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                          for user_vector_j in interest_user_matrix]
                         for user_vector_i in interest_user_matrix]

def most_similar_interests_to(interest_id):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs,
                  key=lambda pair: pair[1],
                  reverse=True)

def item_based_suggestions(user_id, include_current_interests=False):
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_matrix[user_id]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity

    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]


if __name__ == "__main__":


    print()

    print("Подобие (схожесть) на основе пользователя")
    print("наиболее похожие на пользователя 0")
    print(most_similar_users_to(0))

    print("Рекомендации для пользователя 0")
    print(user_based_suggestions(0))
    print()

    print("Коллаборативная фильтрация по схожести предметов")
    print("наиболее похожие на 'Big Data'")
    print(most_similar_interests_to(0))
    print()

    print("Рекомендации для пользователя 0")
    print(item_based_suggestions(0))


Подобие (схожесть) на основе пользователя
наиболее похожие на пользователя 0
[(9, 0.5669467095138409), (1, 0.3380617018914066), (8, 0.1889822365046136), (13, 0.1690308509457033), (5, 0.1543033499620919)]
Рекомендации для пользователя 0
[('MapReduce', 0.5669467095138409), ('MongoDB', 0.50709255283711), ('Postgres', 0.50709255283711), ('NoSQL', 0.3380617018914066), ('neural networks', 0.1889822365046136), ('deep learning', 0.1889822365046136), ('artificial intelligence', 0.1889822365046136), ('databases', 0.1690308509457033), ('MySQL', 0.1690308509457033), ('Python', 0.1543033499620919), ('R', 0.1543033499620919), ('C++', 0.1543033499620919), ('Haskell', 0.1543033499620919), ('programming languages', 0.1543033499620919)]

Коллаборативная фильтрация по схожести предметов
наиболее похожие на 'Big Data'
[('Hadoop', 0.8164965809277261), ('Java', 0.6666666666666666), ('MapReduce', 0.5773502691896258), ('Spark', 0.5773502691896258), ('Storm', 0.5773502691896258), ('Cassandra', 0.4082482904638